## Load necessary packages

In [1]:
import hail as hl
import numpy as np
hl.init()

Running on Apache Spark version 3.1.2
SparkUI available at http://localhost:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.91-44b441376f9a
LOGGING: writing to /Users/taotaotan/Desktop/hail/hail-20220323-1602-0.2.91-44b441376f9a.log


In [2]:
#load in plotting features
from hail.plot import show
hl.plot.output_notebook()

Loading BokehJS ...

<br>
<br>
<br>
<br>

## The following 6 files have been generated, but we will only use 3 of them for Tractor GWAS analysis here

<br>

**ASW.phased.anc0.dosage.txt**  
**ASW.phased.anc0.hapcount.txt**  
ASW.phased.anc0.vcf  
**ASW.phased.anc1.dosage.txt**  
ASW.phased.anc1.hapcount.txt  
ASW.phased.anc1.vcf  

<br>

Please refer to [our tutorial](https://atkinson-lab.github.io/Tractor-tutorial/Local.html) if you want to know the difference between GWAS, Admixture mapping and Tractor.

<br>

We will read `ASW.phased.anc0.dosage.txt`, `ASW.phased.anc0.hapcount.txt`, and `ASW.phased.anc1.dosage.txt` as [hail matrix table](https://hail.is/docs/0.2/overview/matrix_table.html), and `Phe.txt` as [hail table](https://hail.is/docs/0.2/overview/table.html)



In [3]:
row_fields={'CHROM': hl.tstr, 'POS': hl.tint, 'ID': hl.tstr, 'REF': hl.tstr, 'ALT': hl.tstr} 

#####################################################################################
########################### read anc0 hapcount file #################################
anc0hap = hl.import_matrix_table('ADMIX_COHORT/ASW.phased.anc0.hapcount.txt', 
                                row_fields=row_fields, row_key=[], min_partitions=32) 

anc0hap = anc0hap.key_rows_by().drop('row_id')
anc0hap = anc0hap.key_rows_by(locus=hl.locus(anc0hap.CHROM, anc0hap.POS))  




###################################################################################
########################### read anc0 dosage file #################################
anc0dose = hl.import_matrix_table('ADMIX_COHORT/ASW.phased.anc0.dosage.txt', 
                                row_fields=row_fields, row_key=[], min_partitions=32) 

anc0dose = anc0dose.key_rows_by().drop('row_id')
anc0dose = anc0dose.key_rows_by(locus=hl.locus(anc0dose.CHROM, anc0dose.POS))  




###################################################################################
########################### read anc1 dosage file #################################
anc1dose = hl.import_matrix_table('ADMIX_COHORT/ASW.phased.anc1.dosage.txt', 
                                row_fields=row_fields, row_key=[], min_partitions=32) 

anc1dose = anc1dose.key_rows_by().drop('row_id')
anc1dose = anc1dose.key_rows_by(locus=hl.locus(anc1dose.CHROM, anc1dose.POS))  



###################################################################################
############################# read Phenotype file #################################
Phe = hl.import_table('PHENO/Phe.txt', types={'IID': hl.tstr, 'y': hl.tfloat64}).key_by('IID')

# In our Phe.txt file, for simplicity we don’t include any covariates. 
# But you may want to include covariates into your analysis, e.g. admixture proportion (adm)
# You can add more columns (e.g. adm) as follow:

# Phe = hl.import_table('PHENO/Phe.txt', types={'IID': hl.tstr, 'y': hl.tfloat64, 'adm': hl.tfloat64}).key_by('IID')

2022-03-23 16:02:45 Hail: INFO: Reading table without type imputation
  Loading field 'IID' as type str (user-supplied)
  Loading field 'y' as type float64 (user-supplied)


<br>
<br>
<br>
<br>

## Annotate hail matrix table

<br>

We will use `anc0hap` matrix table as our main table. We will annotate columns (samples) with their corresponding phenotypes and covariates (if exit), and annotate each entry with `anc0dose` and `anc1dose`. 

In [4]:
# annotate Phenotype to anc0hap
# you may need to annotate columns again with covariates as well if these are not included in your phenotype file.

anc0hap = anc0hap.annotate_cols(Pheno = Phe[anc0hap.col_id])


In [5]:
anc0hap = anc0hap.annotate_entries(anc0dose = anc0dose[anc0hap.locus, anc0hap.col_id], 
                                   anc1dose = anc1dose[anc0hap.locus, anc0hap.col_id])


2022-03-23 16:02:45 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


<br>
<br>
<br>
<br>

## Run linear regression entry-wise, and append the results to each row (variant)

<br>

To run entry-wise linear regression, we will use `hl.agg.linreg(phenotype, [1.0, X1, X2, X3 ...])`, where `[]` contains all independent variables. It's also worth noticing that Hail expect users to explicitly specify the intercept `1.0`.

#### In our example, we will retrieve the following information to pass to [`hl.agg.linreg()`](https://hail.is/docs/0.2/aggregators.html?highlight=min#hail.expr.aggregators.linreg) function:  
phenotype `anc0hap.Pheno.y` from column annotation  
AFR local ancestries (`X1`) from entry `anc0hap.x`  
AFR risk alleles (`X2`) from entry annotation `anc0hap.anc0dose.x`  
EUR risk alleles (`X3`) from entry annotation `anc0hap.anc1dose.x` 

In [6]:
anc0hap = anc0hap.annotate_rows(
    lm = hl.agg.linreg(
            anc0hap.Pheno.y,
            [1.0, anc0hap.x, anc0hap.anc0dose.x, anc0hap.anc1dose.x]
        )
    )


#################################################################################
############## if you have covariates like admixture proportion #################

# anc0hap = anc0hap.annotate_rows(
#     lm = hl.agg.linreg(
#             anc0hap.Pheno.y,
#             [1.0, anc0hap.x, anc0hap.anc0dose.x, anc0hap.anc1dose.x, anc0hap.Pheno.adm]
#         )
#     )

In [7]:
anc0hap.lm.show()

2022-03-23 16:02:55 Hail: INFO: Coerced sorted dataset
2022-03-23 16:02:56 Hail: INFO: Coerced sorted dataset
2022-03-23 16:02:57 Hail: INFO: Coerced sorted dataset


+---------------+-----------------------------------------+
| locus         | lm.beta                                 |
+---------------+-----------------------------------------+
| locus<GRCh37> | array<float64>                          |
+---------------+-----------------------------------------+
| 22:16050115   | NA                                      |
| 22:16050213   | NA                                      |
| 22:16050783   | NA                                      |
| 22:16050840   | NA                                      |
| 22:16051249   | [1.71e+02,8.80e-02,-2.45e-01,4.42e-01]  |
| 22:16052080   | [1.71e+02,-6.78e-02,1.17e+00,-1.57e+00] |
| 22:16052271   | NA                                      |
| 22:16052463   | [1.71e+02,5.01e-02,-6.23e-01,-7.65e-01] |
| 22:16052684   | NA                                      |
| 22:16052962   | [1.71e+02,9.72e-02,4.55e-01,4.80e-01]   |
+---------------+-----------------------------------------+

+---------------------------------------+
| lm.standard_error                     |
+---------------------------------------+
| array<float64>                        |
+---------------------------------------+
| NA                                    |
| NA                                    |
| NA                                    |
| NA                                    |
| [5.86e-01,3.62e-01,1.96e+00,1.22e+00] |
| [5.47e-01,3.44e-01,1.13e+00,1.41e+00] |
| NA                                    |
| [5.39e-01,3.43e-01,1.97e+00,1.96e+00] |
| NA                                    |
| [5.85e-01,3.60e-01,1.15e+00,1.22e+00] |
+---------------------------------------+

+-----------------------------------------+
| lm.t_stat                               |
+-----------------------------------------+
| array<float64>                          |
+-----------------------------------------+
| NA                                      |
| NA                                      |
| NA                                      |
| NA                                      |
| [2.92e+02,2.43e-01,-1.25e-01,3.63e-01]  |
| [3.13e+02,-1.97e-01,1.03e+00,-1.12e+00] |
| NA                                      |
| [3.17e+02,1.46e-01,-3.17e-01,-3.90e-01] |
| NA                                      |
| [2.92e+02,2.70e-01,3.95e-01,3.94e-01]   |
+-----------------------------------------+

+---------------------------------------+----------------------------+
| lm.p_value                            | lm.multiple_standard_error |
+---------------------------------------+----------------------------+
| array<float64>                        |                    float64 |
+---------------------------------------+----------------------------+
| NA                                    |                         NA |
| NA                                    |                         NA |
| NA                                    |                         NA |
| NA                                    |                         NA |
| [3.52e-92,8.08e-01,9.01e-01,7.18e-01] |                   1.94e+00 |
| [6.76e-94,8.44e-01,3.06e-01,2.69e-01] |                   1.90e+00 |
| NA                                    |                         NA |
| [2.92e-94,8.84e-01,7.52e-01,6.98e-01] |                   1.94e+00 |
| NA                                    |                         NA |
| [3.39e-92,7.88e-01,6.94e-01,6.95e-01] |                   1.94e+00 |
+---------------------------------------+----------------------------+

+-----------------------+-----------------------+-----------+
| lm.multiple_r_squared | lm.adjusted_r_squared | lm.f_stat |
+-----------------------+-----------------------+-----------+
|               float64 |               float64 |   float64 |
+-----------------------+-----------------------+-----------+
|                    NA |                    NA |        NA |
|                    NA |                    NA |        NA |
|                    NA |                    NA |        NA |
|                    NA |   

<br>

The above table provides us with summary statistics for each variant with the following structure `[intercept,anc0hap,anc0dose,anc1dose]`


Take the locus `22:16051249` (the 5th row) as an example, we have effect size as `[1.71e+02,8.80e-02,-2.45e-01,4.42e-01]`. This means:  

Effect size of `intercept` is `1.71e+02`  
Effect size of `anc0hap` is `8.80e-02`  
Effect size of `anc0dose` is `-2.45e-01`  
Effect size of `anc1dose` is `4.42e-01`  

**The order of the results depends on how you annotate your entries!**


<br>  

Also you may have noticed that we have a lot of `NA` output from Hail. This is because we have redundant columns and this causes the calculation to fails. Here are possible reasons:   

1. All local ancestry of this variant is AFR/EUR  
2. Monomorphic variants  
3. And more...

Typically you wouldn't encounter this problem with a larger sample size. We also optimized `RunTractor.py` to reduce this type of problems as much as possible.

<br>
<br>
<br>
<br>

## Draw Manhattan Plot



In [8]:
p = hl.plot.manhattan(anc0hap.lm.p_value[2], 
                      title='AFR', 
                      collect_all=False, 
                      n_divisions = 5000, 
                      significance_line=5e-08) 
show(p)

2022-03-23 16:03:09 Hail: INFO: Coerced sorted dataset
2022-03-23 16:03:10 Hail: INFO: Coerced sorted dataset
2022-03-23 16:03:11 Hail: INFO: Coerced sorted dataset


![AFR](images/AFR.png)

In [9]:
p = hl.plot.manhattan(anc0hap.lm.p_value[3], 
                      title='EUR', 
                      collect_all=False, 
                      n_divisions = 5000, 
                      significance_line=5e-08) 
show(p)

2022-03-23 16:03:21 Hail: INFO: Coerced sorted dataset
2022-03-23 16:03:22 Hail: INFO: Coerced sorted dataset
2022-03-23 16:03:23 Hail: INFO: Coerced sorted dataset


![EUR](images/EUR.png)

<br>

From the Manhattan plot, we have a hit `Chr22:23742105` in AFR track but not in EUR track.

<br>


Here we only analyzed chromosome 22. However, the same code should apply to a genome-wide analysis. 


<br>

We also provide code to run QQ plot, which helps users to examine if p-value inflation exists. In real-world data analysis, It’s always recommended to run this step. However, we didn't draw QQ plot here since the Phenotype data is simulated and our QQ plot isn’t representative. 

In [10]:
# p = hl.plot.qq(anc0hap.lm.p_value[2], title='AFR')
# show(p)